In [1]:
import requests
import base64
import json
import pandas as pd
import logging

base_url = 'https://jira.footlocker.com/rest/api/2/search'

In [2]:

def get_report(base_url):
    logger.info("Getting report results...")
    header_gs = {'Accept': 'application/json'}
    headers = { 'Authorization' : 'Bearer %s' %  'JIRA_TOKEN'}
    # 'Content-Length':3000 }
    logger.info(base_url)
    
    for i in range(1,2):
        if i>1:
            logger.info("Retrying the request again....")   
        r = requests.get(base_url,headers=headers)
        logger.info(r)
        if r.ok:
            logger.info("Report results received...")
            logger.info("HTTP %i - %s" % (r.status_code, r.reason))
            return r.text
        else:
            logger.error("HTTP %i - %s" % (r.status_code, r.text))           
            continue
    return None

In [1]:
def getDataSet(query):
    pd_obj = None
    maxRslt = 2000
    totalRsltFetch = -1
    df= pd.DataFrame()
    while True:
        dataset= get_report(base_url+query+f"&maxResults={maxRslt}&startAt={totalRsltFetch + 1 }")
        if dataset != None:
            obj = json.loads(dataset)
            logger.info(f"total: {obj['total']}")
            totalRsltFetch = totalRsltFetch + maxRslt

            if obj['issues']:
                df = pd.concat([df, pd.json_normalize(obj['issues']) ], ignore_index=True)
            else:
               logger.error(f'No records found....')
            if (totalRsltFetch >= obj['total'] ):
                break
        else:
            logger.error(f'Unable to fetch dataset....')
            break
        # pd_obj = pd.json_normalize(obj['issues'])
            

    return df

In [ ]:
def getUrlDataSet(url):
    pd_obj = None

    dataset= get_report(url)
    if dataset != None:
        pd_obj = json.loads(dataset)
        # pd_obj = pd.json_normalize(obj['issues'])
    return pd_obj

In [ ]:
def setup_logger( level=logging.INFO):
    """Configures a logger and returns it."""

    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

    # handler = logging.FileHandler('./log/soleProj.log')
    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
   
    logger = logging.getLogger()
    logger.setLevel(level)
    
    if (logger.hasHandlers()):
        logger.handlers.clear()

    logger.addHandler(handler)
   

    return logger